<a href="https://colab.research.google.com/github/Alaqeelbasil/traffic-analysis/blob/main/YOLO_Practical_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [151]:
# prompt: library opencv

!pip install opencv-python


In [152]:
!pip install ultralytics
!pip install opencv-p

ERROR: Could not find a version that satisfies the requirement opencv-p (from versions: none)
ERROR: No matching distribution found for opencv-p


## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [153]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [154]:
import cv2
from ultralytics  import YOLO , solutions

## Prepare the Video Capture

In [155]:
model = YOLO('/content/drive/MyDrive/Trainees - المتدربين/Week 6/6- Introduction to computer vision/5- Weekly Project (Sunday)/Dataset/YOLO_Model.pt')

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

## Get Video Information

In [156]:
cap = cv2.VideoCapture('/content/drive/MyDrive/Trainees - المتدربين/Week 6/6- Introduction to computer vision/5- Weekly Project (Sunday)/Dataset/YOLO_Video.mp4')

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

## Prepare Video Writer to Store the Output

In [157]:
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

In [158]:
video_writer = cv2.VideoWriter('Cars_counter.mp4',cv2.VideoWriter_fourcc(*'mp4v'),fps,(frame_width,frame_height))

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [159]:
frame_read = cap.read()[1]

In [ ]:
#defining coordinates
top_roi = (1200, 400, 1300, 1300)
bottom_roi = (300, 500, 700, 600)
left_roi = (270, 450, 650, 550)
right_roi = (900, 200, 1000, 400)

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [ ]:
# vehicle counts
top_count = 0
bottom_count = 0
left_count = 0
right_count = 0

In [ ]:
while cap.isOpened():
    frame = cap.read()
    results = model(frame, stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0] #Extract bounding box coordinates
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            cx = int((x1 + x2) / 2) #Calculate center of vehicle
            cy = int((y1 + y2) / 2)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) #Draw bounding box

            conf = box.conf[0]
            cls = int(box.cls[0])
            cv2.putText(frame, f'Conf: {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(frame, f'ID: {box.id}', (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(frame, f'Class: {cls}', (x1, y1 - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            if (             # this if will Check if vehicle exited the road
                (top_roi[0] <= cx <= top_roi[2] and top_roi[1] <= cy <= top_roi[3])
                and (cy < top_roi[3] / 2)
            ):
                top_count += 1
            elif (
                (bottom_roi[0] <= cx <= bottom_roi[2] and bottom_roi[1] <= cy <= bottom_roi[3])
                and (cy > bottom_roi[3] / 2)
            ):
                bottom_count += 1
            elif left_roi[0] <= cx <= left_roi[2] and left_roi[1] <= cy <= left_roi[3]:
                left_count += 1
            elif right_roi[0] <= cx <= right_roi[2] and right_roi[1] <= cy <= right_roi[3]:
                right_count += 1

#Drawing ROI rectangles
    cv2.rectangle(frame, top_roi[:2], top_roi[2:], (0, 255, 0), 2)
    cv2.rectangle(frame, bottom_roi[:2], bottom_roi[2:], (0, 0, 255), 2)
    cv2.rectangle(frame, left_roi[:2], left_roi[2:], (255, 0, 0), 2)
    cv2.rectangle(frame, right_roi[:2], right_roi[2:], (255, 255, 0), 2)

#calculate the car counting
    cv2.putText(frame, f'Top: {top_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(frame, f'Bottom: {bottom_count}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.putText(frame, f'Left: {left_count}', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
    cv2.putText(frame, f'Right: {right_count}', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

#output the video
    video_writer.write(frame)

cap.release()
video_writer.release()
cv2.destroyAllWindows()


## Save and Submit Your Work

Submit both the notebook and the output video